In [1]:
%matplotlib inline
import sys
import os
DATA_PATH = os.getenv('DATA_PATH')
CODE_PATH = os.getenv('CODE_PATH')
FIGURE_PATH = os.getenv('FIGURE_PATH')

sys.path.insert(0, os.path.join(CODE_PATH))

import pandas as pd
import numpy as np
import json
import re

import time

from src.load import EGRID, BA_DATA

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from calendar import month_abbr

import logging.config
logging.config.fileConfig(os.path.join(CODE_PATH, "src/logging.conf"))
logger = logging.getLogger(__name__)

In [2]:
plt.style.use('seaborn-paper')
plt.rcParams['figure.figsize'] = [6.99, 2.5]
plt.rcParams['grid.color'] = 'k'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams["figure.dpi"] = 200
plt.rcParams["figure.dpi"] = 200
plt.rcParams['font.size'] = 10

import cmocean
cmap = cmocean.cm.cmap_d['phase']
COLORS = sns.color_palette('colorblind')

In [3]:
# Make sure directory exists
os.makedirs(os.path.join(FIGURE_PATH, "si", "sankey"), exist_ok=True)

In [4]:
# Version 2

var = "CO2"
for var in ["CO2", "SO2", "NOX"]:
    poll = BA_DATA(fileNm=os.path.join(DATA_PATH, "analysis/SEED_%s_Y.csv" % var), variable=var)
    poll.df.index=[2016]

    poll_scalings = {"CO2": 1e-6, "SO2": 1e-3, "NOX": 1e-3}
    poll_units = {"CO2": "Mtons", "SO2": "ktons", "NOX": "ktons"}
    poll_scaling = poll_scalings[var]

    # helper functions
    def get_import_export_bas(poll, ba):
        importers = []
        exporters = []
        for ba2 in poll.get_trade_partners(ba):
            if poll.df.loc[:,poll.KEY["ID"]%(ba, ba2)].values[0] < 0:
                importers.append(ba2)
            else:
                exporters.append(ba2)
        return (importers, exporters)

    def get_netP(poll, ba):
        TI_I = -poll.df.loc[:, [poll.KEY["ID"]%(ba, ba2) for ba2 in get_import_export_bas(poll, ba)[0]]].sum(axis=1)
        TI_E = poll.df.loc[:, [poll.KEY["ID"]%(ba, ba2) for ba2 in get_import_export_bas(poll, ba)[1]]].sum(axis=1)
        TI = poll.df.loc[:, poll.KEY["TI"]%ba]
        C = poll.df.loc[:, poll.KEY["D"]%ba]
        P = poll.df.loc[:, poll.KEY["NG"]%ba]
        if (P-TI_E - (C-TI_I)).values[0]>1e-2:
            print("%s: something is wrong" % ba)
        return (P-TI_E).values[0]

    regions = [
        "AVA", "AZPS","BANC", "BPAT","CHPD",
        "CISO", "DEAA", "DOPD", "EPE", "GCPD", "GRMA", "GWA",
        "HGMA", "IID", "IPCO", "LDWP", "NEVP", "NWMT",
        "PACE", "PACW", "PGE", "PNM", "PSCO",
        "PSEI", "SCL", "SRP", "TEPC","TIDC",
        "TPWR", "WACM", "WALC", "WAUW", "WWA"]
    exclude_bas = ["SCL", "CHPD", "DOPD", "GCPD", "TPWR",
                   "WAUW", "TIDC", "PACW", "PGE", "PSEI",
                  "IID", 'AVA', 'DEAA', 'GWA', 'HGMA', 'WWA']
    regions = [ba for ba in regions if ba not in exclude_bas]

    graph = {}

    links = []

    wierdos = []  # BAs with negative netP
    for ba in regions:
        netP = get_netP(poll, ba)
        if netP > 0 :
            links += [{"source":ba+" (P)", "target":ba+" (C)",
                               "value":netP * poll_scaling}]
        else:
            wierdos += [ba]
            print("%s: netP < 0" % ba)

    for i in range(len(regions)):
        for j in range(i, len(regions)):
            from_ba = regions[i]
            to_ba = regions[j]
            if ((poll.KEY["ID"] % (from_ba, to_ba) in poll.df.columns)
                    & (poll.KEY["ID"] % (to_ba, from_ba) in poll.df.columns)):
                co2_transfer = poll.df.loc[:, poll.KEY["ID"] % (from_ba, to_ba)].values[0]
                if co2_transfer < 0: # Have this be positive
                    from_ba = regions[j]
                    to_ba = regions[i]
                    co2_transfer = poll.df.loc[:, poll.KEY["ID"] % (from_ba, to_ba)].values[0]
                src_txt = "%s (P)" % from_ba
                if from_ba in wierdos:
                    src_txt = "%s* (P)" % from_ba
                targ_txt = "%s (C)" % to_ba
                if to_ba in wierdos:
                    targ_txt = "%s* (C)" % to_ba
                links += [{"source": src_txt, "target": targ_txt,
                           "value": co2_transfer * poll_scaling}]

    graph["links"] = links

    nodes = []
    for link in links:
        nodes.append(link["source"])
        nodes.append(link["target"])

    graph["nodes"] = [{"name":n} for n in np.unique(nodes)]

    graph["units"] = poll_units[var]

    graph["totalD"] = poll.df.loc[:, poll.get_cols(regions, "D")].sum().sum() * poll_scaling
    graph["totalG"] = poll.df.loc[:, poll.get_cols(regions, "NG")].sum().sum() * poll_scaling
    graph["var"] = var
    
    graphPath_out = os.path.join("sankey", "data", "data_%s.json" % var)
    with open(graphPath_out, 'w') as fw:
        json.dump(graph, fw)

    # Also save a copy to the FIGURE_PATH folder
    graphPath_out2 = os.path.join(FIGURE_PATH, "si", "sankey", "data_%s.json" % var)
    with open(graphPath_out2, 'w') as fw:
        json.dump(graph, fw)

WALC: netP < 0
BANC: netP < 0
BPAT: netP < 0
IPCO: netP < 0
LDWP: netP < 0
WALC: netP < 0
WALC: netP < 0


In [5]:
poll.df.loc[:, poll.get_cols(regions, "D")].sum().sum() * poll_scaling

228.76443141715646

# Add in production and consumption.
To do that, need to calculate $production - exports$ and $consumption -imports$. Recalculate TI_E and TI_I to do this

In [6]:
poll.df.loc[:,poll.get_cols(regions, "NG")].sum().sum() * poll_scaling

233.90365119949738

In [7]:
poll.df.loc[:,poll.get_cols(regions+exclude_bas, "NG")].sum().sum() * poll_scaling

241.55093978534333

In [8]:
print("Fraction of %s emitted: %.2f" % (var,
    100*poll.df.loc[:,poll.get_cols(regions, "NG")].sum().sum()
    / poll.df.loc[:,poll.get_cols(regions+exclude_bas, "NG")].sum().sum()))

Fraction of NOX emitted: 96.83


In [9]:
ba = "CISO"
TI_I = -poll.df.loc[:, [poll.KEY["ID"]%(ba, ba2) for ba2 in get_import_export_bas(poll, ba)[0]]].sum(axis=1)
TI_E = poll.df.loc[:, [poll.KEY["ID"]%(ba, ba2) for ba2 in get_import_export_bas(poll, ba)[1]]].sum(axis=1)
TI = poll.df.loc[:, poll.KEY["TI"]%ba]
C = poll.df.loc[:, poll.KEY["D"]%ba]
P = poll.df.loc[:, poll.KEY["NG"]%ba]

In [10]:
TI_I * poll_scaling

2016    18.223121
dtype: float64

In [11]:
TI_E * poll_scaling

2016    0.410319
dtype: float64

In [12]:
P * poll_scaling

2016    40.846392
Name: NOX_CISO_NG, dtype: float64

In [13]:
C * poll_scaling

2016    58.659194
Name: NOX_CISO_D, dtype: float64